In [8]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from datetime import datetime

from bs4 import BeautifulSoup 
import requests
import pprint
import os


In [10]:
url = "https://www.bowtie.com.hk/zh/offer"
driver = webdriver.Chrome()
driver.get(url)

data = []

elements = driver.find_elements(By.CLASS_NAME, "offer--text")
for body in elements:
    end_date = body.find_elements(By.CLASS_NAME, "offer--promo-item")
    end_date = end_date[-1].text.replace("\n", "")
    product_type = body.find_element(By.CLASS_NAME, "ts--body.offer--cate.tc--grey").text
    product_type = product_type.strip().lstrip()
    promotion_title = body.find_element(By.CLASS_NAME, "offer--title.tc--black.ts--h2").text
    promotion_text =  body.find_element(By.CSS_SELECTOR, "p.ts--body").text

    link_element = body.find_element(By.CLASS_NAME, "w-inline-block.btn--main.btn--outline")
    if link_element.text:
        link = link_element.get_attribute("href")
    
        data.append({
            "end_date": end_date,
            "product_type": product_type,
            "promotion_title": promotion_title,
            "promotion_text": promotion_text,
            "link": link
        })
    link_element= None



# finally:
driver.quit()

In [11]:
df = pd.DataFrame(data)
df

,end_date,product_type,promotion_title,promotion_text,link
0,期限：即日起至 2024 年 8 月 14 日,自願醫保｜人壽保｜戰癌保｜危疾保｜觸木保,新客戶投保指定產品享首年保費低至 65 折,新客戶投保指定產品享首年保費低至 65 折。,https://app.bowtie.com.hk/product-shelf/?coupo...
1,期限：即日起至 2024 年 9 月 30 日,自願醫保,轉嚟 Bowtie 自願醫保享首年保費 5 折,你份醫保保費愈加愈貴!? Bowtie 致力打造長遠都可負擔嘅產品，Keep 住幫你慳！,https://www.bowtie.com.hk/offer/switching-page
2,期限：即日起至 2024 年 8 月 14 日,自願醫保,首年保費 65 折 + 每人額外享 $1000 BowtieCash 獎賞,家人的健康是生命中最寶貴的財產之一。Bowtie 照顧你和家人每個人生階段的健康，提供性價比...,https://www.bowtie.com.hk/zh/offer/family-prom...
3,期限：即日起至 2024 年 8 月 31 日,個人保險產品,與親友齊享限時優惠 推薦人數不設上限,每宗成功推薦，享高達 5 個月保費豁免。,https://www.bowtie.com.hk/zh/offer/referral-pr...
4,期限：即日起至 2024 年 8 月 31 日,個人保險產品,Bowtie 現有客戶專享低至 3 折,成功為自己或家人首次投保其他個人保險產品，可享有指定優惠。,https://www.bowtie.com.hk/offer/bowtie-vhis-ex...
5,期限：即日起至 2024 年 9 月 30 日,自願醫保,照 CT／MRI／PET-CT 享超市現金券,"預先批核申請一旦通過，受保人除了可享用「免索償服務」外，更可獲高達 $2,000 超市現金券。",https://www.bowtie.com.hk/offer/health-special...


In [21]:
url = df['link'][0]
driver = webdriver.Chrome()
driver.get(url)

link_element = driver.find_elements(By.CSS_SELECTOR, "a.MuiTypography-root.MuiLink-root")
terms_link = link_element[-1].get_attribute("href")
terms_page = link_element[-1].click()
time.sleep(2)

# Switch to the newly opened tab
driver.switch_to.window(driver.window_handles[1])

# Find elements on the new page
terms_page = driver.find_elements(By.CLASS_NAME, "ts--body-lg.w-richtext")

data_terms = []

for element in terms_page:
    date = element.find_element(By.TAG_NAME, "p").text
    discount_detail = element.find_element(By.TAG_NAME, 'li').text
    
    data_terms.append({
        "date": date,
        "discount_detail": discount_detail
    })
data_switch = []
url = 'https://www.bowtie.com.hk/tnc/vhis-switching'
driver.get(url)
time.sleep(2)
body = driver.find_element(By.CLASS_NAME,"ts--body-2.w-richtext")
date = body.find_element(By.TAG_NAME, "p").text
discount_detail = body.find_element(By.TAG_NAME, 'li').text

data_switch.append({
    "date": date,
    "discount_detail": discount_detail
})

data_ref = []

url = "https://www.bowtie.com.hk/static/friends-and-family-referral-program"
driver = webdriver.Chrome()
driver.get(url)

body = driver.find_element(By.CLASS_NAME,"ts--body-lg.w-richtext")
date = body.find_elements(By.TAG_NAME, "p")[2].text
body_ls = body.find_elements(By.TAG_NAME, "ul")[1]
body_details = body_ls.find_elements(By.TAG_NAME, "li")
for detail in body_details:
    product, referrer, referee = detail.text.split("\n")
    data_ref.append({
        "product_name": product,
        "referrer_name": referrer,
        "referee_name": referee
    })
data_exist = []

url = "https://www.bowtie.com.hk/offer/existing-customer-offer"
driver = webdriver.Chrome()
driver.get(url)

details = driver.find_element(By.CLASS_NAME,"ts--footnote-2").text
data_exist.append({
    "details": details
})

data_imaging = []

url = "https://www.bowtie.com.hk/offer/health-specialist-imaging"
driver.get(url)
body = driver.find_element(By.CLASS_NAME, "imaging-table-wrapper")
title = body.find_element(By.CLASS_NAME, "sp--mb-3.ts--h2").text
details = body.find_element(By.CLASS_NAME, "ts--body.txt--center").text
table = body.find_element(By.CLASS_NAME, "ts--body.bgc--white.ref--tabe-wrap.sp--mb-4")
rows = table.find_elements(By.CLASS_NAME, "ref--table-row")
for row in rows:
    col = row.find_elements(By.CLASS_NAME, "ref--table-col")
    left_col = col[0].text
    right_col = col[1].text
    data_imaging.append(
        {
        "title": title,
        "details": details,
        "left_column": left_col,
        "right_column": right_col   
        }
    )

data_family = []

url = "https://www.bowtie.com.hk/zh/offer/family-promotion"
driver = webdriver.Chrome()
driver.get(url)
body = driver.find_element(By.CLASS_NAME, "new-hero--text")
product = body.find_element(By.CLASS_NAME, "ts--h4.sp--mb-1").text
summary = body.find_element(By.CLASS_NAME, "ts--body.sp--mb-4").text

details = driver.find_element(By.CLASS_NAME, "sp--mb-2.txt--center")
date = details.find_element(By.CLASS_NAME, "ts--body").text
detail_1 = details.find_element(By.CLASS_NAME, "ts--h2.tc--pink").text
detail_2 = details.find_element(By.CLASS_NAME, "ts--h3").text
detail_3 = details.find_elements(By.CLASS_NAME, "ts--h2.tc--pink")[1].text
full_detail = detail_1 + " " + detail_2 + detail_3

data_family.append({
    "date": date,
    "product": product,
    "summary": summary,
    "detail": full_detail
})
driver.quit()   


In [35]:
data_family

[{'date': '2024 年 8 月 14 日前',
  'product': 'Bowtie 自願醫保',
  'summary': '限時家庭優惠：\n首年保費 65 折 + 每人額外享 $1000 BowtieCash 獎賞',
  'full_detail': '首年保費 65 折 2 人或以上成功投保 每人額外再賺$1,000 BowtieCash'}]

In [38]:
df_terms = pd.DataFrame(data_terms)
df_switch = pd.DataFrame(data_switch)
df_ref = pd.DataFrame(data_ref)
df_switch = pd.DataFrame(data_switch)
df_imaging = pd.DataFrame(data_imaging)
df_exist = pd.DataFrame(data_exist)
df_family = pd.DataFrame(data_family)

In [23]:
data_terms
df_terms = pd.DataFrame(data_terms)

In [39]:
rundate = datetime.now().strftime("%Y%m%d")

parent_dir = os.getcwd()
directory = "Bowtie_promotion"
path = os.path.join(parent_dir, directory)

with pd.ExcelWriter(f"{parent_dir}/{directory}_{rundate}.xlsx", engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name= "summary", index= False)
    df_terms.to_excel(writer, sheet_name= "new_customer_discount", index=False)
    df_switch.to_excel(writer, sheet_name= "switching_customer", index=False)
    df_family.to_excel(writer, sheet_name= "family_discount", index= False)
    df_ref.to_excel(writer, sheet_name= "referral_discount", index= False)
    df_exist.to_excel(writer, sheet_name= "existing_customer", index=False)
    df_imaging.to_excel(writer, sheet_name= "imaging_discount", index= False)
    
    